In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio
from datetime import datetime
from pytz import timezone, utc
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap
from pytz import timezone


In [12]:
# --- Parameters ---
original_root = "/Volumes/External/TJ_SAR/02_preprocessed/01_JunethroughDec2024"
mask_root = "/Volumes/External/TJ_SAR/04_iseg"
output_plot_dir = "/Volumes/External/TJ_SAR/05_viz/_20222025"
met_csv_path = "/Volumes/External/TJ_estuary/analysis/TJRTLMET.csv"


In [13]:
pacific = timezone("US/Pacific")
# --- Load meteorological data (in Pacific Time) ---
met_df = pd.read_csv(met_csv_path)
met_df.columns = met_df.columns.str.strip()

met_df["DateTimeStamp"] = pd.to_datetime(
    met_df["DateTimeStamp"], format="%m/%d/%y %H:%M", errors="coerce"
)
met_df["DateTimeStamp"] = met_df["DateTimeStamp"].dt.tz_localize(
    pacific,
    ambiguous="NaT",     # handle fall-back (ambiguous) times
    nonexistent="NaT"    # handle spring-forward (nonexistent) times
)
met_df = met_df.dropna(subset=["DateTimeStamp"])

met_df = met_df.sort_values("DateTimeStamp")


In [14]:
def extract_datetime_from_filename(filename):
    """Extract UTC datetime from Sentinel-1 filename."""
    parts = filename.split("_")
    for part in parts:
        if part.startswith("20") and "T" in part:
            try:
                return utc.localize(datetime.strptime(part, "%Y%m%dT%H%M%S"))
            except ValueError:
                continue
    return None

def get_nearest_met_data(local_time):
    """Find the closest met record to the given local time."""
    time_deltas = (met_df["DateTimeStamp"] - local_time).abs()
    nearest_idx = time_deltas.idxmin()
    row = met_df.loc[nearest_idx]
    return row["WSpd"], row["Wdir"], row["DateTimeStamp"]

def match_original_path(mask_filename):
    return os.path.join(original_root, mask_filename.replace("_JPL0.4_VVDR_cumulative_mask.tif", ".tif"))

In [16]:
import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import math
from matplotlib.colors import Normalize
import matplotlib.patches as patches

def plot_overlay(original_path, mask_path, output_path):
    # --- Extract and convert image time ---
    image_time_utc = extract_datetime_from_filename(os.path.basename(original_path))
    image_time_local = image_time_utc.astimezone(pacific)

    # --- Match meteorological data ---
    wspd, wdir, met_time = get_nearest_met_data(image_time_local)

    # --- Read images ---
    with rasterio.open(original_path) as src:
        original = src.read(1).astype(float)
        bounds = src.bounds
        extent = [bounds.left, bounds.right, bounds.bottom, bounds.top]

    with rasterio.open(mask_path) as src:
        mask = src.read(1)

    # --- Linear 2–98 percentile stretch ---
    # Compute the 2nd and 98th percentiles and linearly map to [0,1]
    vmin, vmax = np.nanpercentile(original, (2, 95))
    clipped = np.clip(original, vmin, vmax)
    normed = (clipped - vmin) / (vmax - vmin)

    # --- Set up side-by-side figure ---
    fig, axes = plt.subplots(1, 2, figsize=(9, 12), constrained_layout=True)

    # Left: original stretched
    axes[0].imshow(normed, cmap="gray", extent=extent)
    axes[0].set_title("Original (2–98% Stretch)", fontsize=12)
    axes[0].axis('off')

    # Right: original + mask
    axes[1].imshow(normed, cmap="gray", extent=extent)

    # Build RGBA overlay for plume mask
    mask_bin = (mask > 0)
    overlay = np.zeros((mask.shape[0], mask.shape[1], 4))
    overlay[mask_bin] = [1.0, 1.0, 0.8, 1.0]   # light yellow, fully opaque

    axes[1].imshow(overlay, extent=extent)
    axes[1].set_title("Original with Plume Mask", fontsize=12)
    axes[1].axis('off')

    # --- Bottom annotation: wind info + local time ---
    wind_text = (
        f"Wind: {wspd:.1f} m/s @ {wdir:.0f}°\n"
        f"Local Time: {image_time_local.strftime('%Y-%m-%d %H:%M')}"
    )
    fig.text(0.5, 0.15, wind_text, ha='center', fontsize=13, fontweight='bold')

    # --- Wind vector in real coordinates ---
    wind_angle_deg = (270 - wdir) % 360
    angle_rad = math.radians(wind_angle_deg)
    height = bounds.top - bounds.bottom
    base_frac = 0.07
    scale = np.clip(wspd / 10, 0.5, 1.5)
    arrow_length = height * base_frac * scale
    dx = arrow_length * math.cos(angle_rad)
    dy = arrow_length * math.sin(angle_rad)
    x0 = bounds.right - 0.1 * (bounds.right - bounds.left)
    y0 = bounds.bottom + 0.05 * height

    axes[1].add_patch(
        patches.FancyArrow(
            x0, y0, dx, dy,
            width=arrow_length * 0.05,
            head_width=arrow_length * 0.15,
            head_length=arrow_length * 0.15,
            length_includes_head=True,
            transform=axes[1].transData,
            color='lime',
            alpha=0.9
        )
    )

    # --- Save plot ---
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"Saved: {output_path}")



In [17]:
# --- Main processing loop ---
for root, _, files in os.walk(mask_root):
    for file in files:
        if file.startswith('.'):
            continue
        if file.endswith("_JPL0.4_VVDR_cumulative_mask.tif"):
            mask_path = os.path.join(root, file)
            original_path = match_original_path(file)

            if os.path.exists(original_path):
                flat_name = file.replace("_JPL0.4_VVDR_cumulative_mask.tif", "_overlay.png")
                output_path = os.path.join(output_plot_dir, flat_name)
                plot_overlay(original_path, mask_path, output_path)
            else:
                print(f"[!] Original image not found for: {file}")

print("Done!")


[!] Original image not found for: S1A_IW_GRDH_1SDV_20221121T015015_20221121T015044_045986_0580C5_7D6E_pre_JPL0.4_VVDR_cumulative_mask.tif
[!] Original image not found for: S1A_IW_GRDH_1SDV_20221121T135317_20221121T135342_045993_058110_4619_pre_JPL0.4_VVDR_cumulative_mask.tif
[!] Original image not found for: S1A_IW_GRDH_1SDV_20221128T134454_20221128T134519_046095_05847D_4A73_pre_JPL0.4_VVDR_cumulative_mask.tif
[!] Original image not found for: S1A_IW_GRDH_1SDV_20221203T015015_20221203T015044_046161_0586BC_72DC_pre_JPL0.4_VVDR_cumulative_mask.tif
[!] Original image not found for: S1A_IW_GRDH_1SDV_20221203T135317_20221203T135342_046168_058705_6F82_pre_JPL0.4_VVDR_cumulative_mask.tif
[!] Original image not found for: S1A_IW_GRDH_1SDV_20221210T134454_20221210T134519_046270_058A79_9DD5_pre_JPL0.4_VVDR_cumulative_mask.tif
[!] Original image not found for: S1A_IW_GRDH_1SDV_20221215T015014_20221215T015043_046336_058CB1_E040_pre_JPL0.4_VVDR_cumulative_mask.tif
[!] Original image not found for: 